<a href="https://colab.research.google.com/github/sagar431/Mystral_LLM/blob/main/Mistral_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-83fxni8r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-83fxni8r
  Resolved https://github.com/huggingface/transformers to commit 3cefac1d974db5e2825a0cb2b842883a628be7a0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

data = load_dataset("OdiaGenAI/odia_context_qa_98k", split="train")
data_df = data.to_pandas()
data_df = data_df[:60000]
data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "###Human: " + x["instruction"] + " " + x["input"] + " ###Assistant: "+ x["output"], axis=1)
data = Dataset.from_pandas(data_df)


tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token


quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                            "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                            quantization_config=quantization_config_loading,
                            device_map="auto"
                        )


model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)


training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-odia-60k",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=3,
        max_steps=250,
        fp16=True,
        push_to_hub=True
)


trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
)


trainer.train()

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the diff

Step,Training Loss
100,0.766700
200,0.626700


TrainOutput(global_step=250, training_loss=0.6790708236694336, metrics={'train_runtime': 3134.363, 'train_samples_per_second': 0.638, 'train_steps_per_second': 0.08, 'total_flos': 848818077696000.0, 'train_loss': 0.6790708236694336, 'epoch': 0.03})

In [10]:
! cp -r /content/mistral-finetuned-alpaca/content/drive/MyDrive

cp: missing destination file operand after '/content/mistral-finetuned-alpaca/content/drive/MyDrive'
Try 'cp --help' for more information.


In [19]:
ls

drive/                     mistral-finetuned-odia/      sample_data/
mistral-finetuned-alpaca/  mistral-finetuned-odia-60k/


In [18]:
!cp /content/mistral-finetuned-odia-60k/content/destination_directory/

cp: missing destination file operand after '/content/mistral-finetuned-odia-60k/content/destination_directory/'
Try 'cp --help' for more information.


# Inference

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-odia-60k")

inputs = tokenizer("""###Human: ପାଲିଓଜୋଇକ ଯୁଗର ଶେଷଭାଗରୁ ଆରମ୍ଭ କରି ଆଣ୍ଟାର୍କଟିକା ଗଠନ କେବେ ଶେଷ ହେଲା? ###Assistant: """, return_tensors="pt").to("cuda")

In [5]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-odia-60k",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

In [6]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

In [7]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

###Human: ପାଲିଓଜୋଇକ ଯୁଗର ଶେଷଭାଗରୁ ଆରମ୍ଭ କରି ଆଣ୍ଟାର୍କଟିକା ଗଠନ କେବେ ଶେଷ ହେଲା? ###Assistant: ଆରମ୍ଭ କରି ଆଣ୍ଟାର୍କଟିକା ଗଠନ କରି ଆରମ୍ଭ �
12.569499254226685


In [14]:
from google.colab import userdata

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('huggingface')

In [12]:
new_model="Odia_mistral_fine_tuning"

In [15]:
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

FileNotFoundError: ignored